In [ ]:
#Import relevant modules.
import pygsti
import numpy as _np

import matplotlib.pyplot as plt
%matplotlib inline

import time
import os
import pickle

basedir = 'D:/measuring/analysis/notebooks/Gateset tomography'

In [ ]:
#Create maximum lengths list
maxLengths = [0,1,2,4,8,16,32]

pickle.dump( maxLengths, open(os.path.join(basedir,"MyMaxLengths.pkl"), "wb"))

In [ ]:
#Construct a target gateset
gs_target = pygsti.construction.build_gateset([2],[('Q0',)], [u'Gx',u'Gy',u'Gu',u'Gv'], 
                                             [ "X(pi/2,Q0)", "Y(pi/2,Q0)","X(pi,Q0)","Y(pi,Q0)"],
                                             prepLabels=['rho0'], prepExpressions=["0"],
                                             effectLabels=['E0'], effectExpressions=["1"], 
                                             spamdefs={'plus': ('rho0','E0'), 'minus': ('rho0','remainder') } )

pygsti.io.write_gateset(gs_target, os.path.join(basedir,"MyTargetGateset.txt") )

In [ ]:
# Load in fiducials, germs, maxLengths:
measFidList = pygsti.io.load_gatestring_list(os.path.join(basedir,"MeasFiducials.txt"))
prepFidList = pygsti.io.load_gatestring_list(os.path.join(basedir,"PrepFiducials.txt"))
germList = pygsti.io.load_gatestring_list(os.path.join(basedir,"MyGerms.txt"))
maxLengths = pickle.load( open(os.path.join(basedir,"MyMaxLengths.pkl")))

#Create a list of GST experiments for this gateset, with
#the specified fiducials, germs, and maximum lengths
listOfExperiments = pygsti.construction.make_lsgst_experiment_list(gs_target.gates.keys(), prepFidList, measFidList, germList, maxLengths)

#Create an empty dataset file, which stores the list of experiments
#plus extra columns where data can be inserted
pygsti.io.write_empty_dataset(os.path.join(basedir,"MyDataTemplate.txt"), listOfExperiments,
                              "## Columns = plus count, count total")

In [ ]:
#Create a gateset of depolarized gates and SPAM relative to target, and generate fake data using this gateset.
gs_datagen = gs_target.depolarize(gate_noise=0.05, spam_noise=0.001)
ds = pygsti.construction.generate_fake_data(gs_datagen, listOfExperiments, nSamples=10000,
                                            sampleError="binomial", seed=2015)

#Save our dataset
pygsti.io.write_dataset(os.path.join(basedir,"MyDataset.txt"), ds)

In [ ]:
#Note; to load the dataset back again, do:
#ds = pygsti.io.load_dataset("tutorial_files/MyDataset.txt")

gs_target.set_all_parameterizations("TP")
results = pygsti.do_long_sequence_gst(os.path.join(basedir,"MyDataset.txt"), gs_target, 
                                        prepFidList, measFidList, germList, maxLengths,
                                        gaugeOptParams={'itemWeights': {'spam': 1e-3, 'gates': 1.0}})

s = pickle.dumps(results)
r2 = pickle.loads(s)
print(r2.gatesets['final estimate'])

In [ ]:
results.create_brief_report_pdf(confidenceLevel=95, filename=os.path.join(basedir,"easy_brief.pdf"), verbosity=2)


In [ ]:
#create a full GST report (most detailed and pedagogical; best for those getting familiar with GST)
results.create_full_report_pdf(confidenceLevel=95, filename=os.path.join(basedir,"easy_full.pdf"), verbosity=2)